## Exploratory Data Analysis

Here we will analyze the car advertisement dataset.  Our goal in this project is to develop and deploy a web application based on this car advertisement dataset.
In this section we will dive deeper into the data so we can determine what types of plots, histograms and charts will give us the most useful information for our web application.

In [1]:
#load the libraries
import pandas as pd
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
import plotly_express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
vehicles_df = pd.read_csv('vehicles_us.csv')

In [3]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
vehicles_df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
1873,5995,2012.0,chevrolet impala,excellent,6.0,gas,132000.0,automatic,sedan,silver,NaN,2019-01-28,12
43320,15500,2011.0,ford f250 super duty,excellent,NaN,diesel,223000.0,automatic,truck,white,1.0,2018-12-31,81
36517,6500,1981.0,chevrolet silverado,good,8.0,gas,28000.0,automatic,truck,brown,NaN,2018-10-15,5
4969,4999,2008.0,ford focus se,excellent,4.0,gas,76300.0,automatic,sedan,red,NaN,2018-07-31,32
10487,5995,2006.0,honda civic,excellent,4.0,hybrid,78000.0,automatic,sedan,white,NaN,2018-12-20,11
50787,4800,2004.0,chevrolet silverado 2500hd,good,8.0,gas,196000.0,automatic,truck,white,1.0,2018-06-03,14
17158,12299,2018.0,nissan sentra,excellent,4.0,gas,26360.0,automatic,other,silver,NaN,2018-11-14,19
39886,5950,2010.0,toyota corolla,excellent,4.0,gas,124500.0,automatic,sedan,grey,NaN,2018-08-25,30
19626,17999,2016.0,nissan maxima,excellent,6.0,gas,43000.0,automatic,sedan,grey,NaN,2018-10-02,72
38767,25995,2015.0,chevrolet suburban,good,8.0,gas,120318.0,automatic,SUV,brown,NaN,2018-09-10,109


Above we have imported our dataset and have gotten an idea of what is in the dataset and where there may be some missing data that we want to look into. For example, there are missing values in paint color that we can fill in as unknown, 4wd is a 1 if it has it but blank if it does not.  We can fill this in with a 0.  We are also missing some model years and odometer readings. These values we may want to dig into.  If we dig in and see that missing odometer readings mean there are 0 miles put on the vehicle that is information we would want to know for our analysis. The make year may also affect our data.  We will also want to change the data type of date posted to datetime.

In [5]:
vehicles_df['date_posted']= pd.to_datetime(vehicles_df['date_posted'], format= '%Y-%m-%d')
print(vehicles_df['date_posted'].dtype) #confirms we changed the datatype

datetime64[ns]


In [6]:
vehicles_df['paint_color'] = vehicles_df['paint_color'].fillna('unknown')

In [7]:
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].fillna(0)

In [8]:
print(vehicles_df[vehicles_df['odometer'].isna()])

       price  model_year                model  condition  cylinders fuel  \
3       1500      2003.0           ford f-150       fair        8.0  gas   
15     17990      2013.0             ram 1500  excellent        8.0  gas   
23      7500      2004.0        jeep wrangler       good        6.0  gas   
24      3950      2009.0         chrysler 200  excellent        4.0  gas   
25     11499      2017.0     chevrolet malibu   like new        4.0  gas   
...      ...         ...                  ...        ...        ...  ...   
51498  15900      2011.0      gmc sierra 1500  excellent        8.0  gas   
51507  29500      2012.0        toyota tundra       good        8.0  gas   
51516   8999      2011.0  jeep grand cherokee       good        6.0  gas   
51519  11750      2015.0         honda accord  excellent        4.0  gas   
51524   6300      2014.0        nissan altima       good        4.0  gas   

       odometer transmission    type paint_color  is_4wd date_posted  \
3           NaN

Here we can see that there is no pattern with the vehicles with unknown odometer readings that could lead us to believe that they have not been used at all.  Instead of filling this values with 0.00 for our calculations, we can use the mean of the other odometer readings.

In [9]:
vehicles_df['odometer'] = vehicles_df['odometer'].fillna(vehicles_df['odometer'].mean())

In [10]:
print(vehicles_df[vehicles_df['model_year'].isna()])

       price  model_year                       model  condition  cylinders  \
1      25500         NaN                  ford f-150       good        6.0   
20      6990         NaN             chevrolet tahoe  excellent        8.0   
65     12800         NaN                  ford f-150  excellent        6.0   
69      7800         NaN                  ford f-150   like new        8.0   
72      3650         NaN              subaru impreza  excellent        NaN   
...      ...         ...                         ...        ...        ...   
51464   8890         NaN        ford f250 super duty       good        8.0   
51465  34595         NaN                   gmc yukon  excellent        8.0   
51487   1495         NaN                  ford f-150       fair        6.0   
51488  21900         NaN  chevrolet silverado 3500hd  excellent        8.0   
51508   4950         NaN     chrysler town & country  excellent        6.0   

         fuel  odometer transmission      type paint_color  is_

Here we can see the same thing as we did with the odometer, so we will fill the missing values with a value of unknown. We likely will not be doing any calculations with these years so it is okay that they are changed to a float type that we can group data by. 

In [11]:
vehicles_df['model_year'] = vehicles_df['model_year'].fillna(vehicles_df['model_year'].mean())

In [12]:
#check there are no missing values again
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    51525 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(6)
memory usage: 5.1+ MB


In [13]:
vehicles_df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
46269,4995,2005.0,toyota highlander,good,NaN,gas,230000.000000,automatic,SUV,unknown,1.0,2019-01-26,25
1323,3500,1998.0,ford ranger,good,6.0,gas,115553.461738,manual,truck,unknown,1.0,2018-08-09,31
4477,2800,1990.0,chevrolet silverado,good,8.0,gas,109000.000000,automatic,pickup,blue,1.0,2019-01-06,32
19037,7295,2016.0,chevrolet cruze,excellent,4.0,gas,105305.000000,automatic,sedan,unknown,0.0,2019-02-25,38
34246,24637,2014.0,ram 1500,excellent,8.0,gas,115553.461738,automatic,truck,unknown,1.0,2018-07-30,71
29277,16500,2017.0,chevrolet equinox,excellent,4.0,gas,37508.000000,automatic,SUV,grey,0.0,2018-08-04,13
12570,6900,2011.0,ford escape,good,6.0,gas,115553.461738,automatic,SUV,white,0.0,2018-05-11,12
36513,11000,2012.0,toyota prius,like new,4.0,hybrid,115553.461738,automatic,sedan,unknown,0.0,2018-07-14,59
3825,9500,1989.0,jeep cherokee,excellent,6.0,gas,101000.000000,automatic,SUV,unknown,1.0,2019-03-05,37
27557,8900,2016.0,nissan versa,good,4.0,gas,77140.000000,automatic,sedan,unknown,0.0,2018-06-09,29


Now we can work on making some charts and comparisons for our web app to help folks view the data when they are making their car buying decisions.

In [14]:
vehicles_df['manufacturer'] = vehicles_df['model'].apply(lambda x: x.split()[0])
vehicles_df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
30165,19995,2011.00000,ford f150,good,NaN,gas,150103.0,automatic,truck,white,1.0,2018-06-04,38,ford
26077,15998,2017.00000,chevrolet malibu,excellent,4.0,gas,24631.0,automatic,sedan,custom,0.0,2019-02-12,52,chevrolet
3200,2900,2008.00000,nissan sentra,excellent,4.0,gas,137179.0,automatic,sedan,silver,0.0,2018-10-13,20,nissan
24889,25500,2016.00000,ford f150,excellent,NaN,gas,26000.0,automatic,pickup,black,1.0,2018-10-31,7,ford
46778,10900,2013.00000,honda cr-v,excellent,4.0,gas,104017.0,automatic,SUV,brown,0.0,2019-03-13,48,honda
29102,4000,1997.00000,toyota tacoma,fair,6.0,gas,324000.0,automatic,truck,white,1.0,2018-06-02,49,toyota
14431,4650,2004.00000,ram 1500,good,8.0,gas,223000.0,automatic,truck,grey,0.0,2018-10-26,11,ram
12706,9900,2009.75047,cadillac escalade,good,8.0,gas,130000.0,automatic,SUV,black,1.0,2019-04-10,44,cadillac
43130,8500,1983.00000,chevrolet suburban,good,8.0,diesel,93225.0,automatic,SUV,blue,1.0,2018-05-10,24,chevrolet
28105,1200,2000.00000,ford f250,good,8.0,gas,193000.0,automatic,pickup,blue,0.0,2019-02-10,33,ford


In [15]:
#Example from blog post 

color_map = {
    'SUV': 'blue',
    'pickup': 'green',
    'sedan': 'red',
    'truck': 'gray',
    'coupe': 'tan',
    'van': 'black',
    'convertible': 'orange',
    'hatchback': 'purple',
    'wagon': 'pink',
    'mini-van': 'yellow',
    'bus': 'brown',
    'offroad': 'cyan',
    'other': 'magenta'
}

fig = px.histogram(vehicles_df, x='manufacturer', color='type', title='Number of vehicles by manufacturer and type', color_discrete_map=color_map)
fig.show()

In [16]:
# Define price ranges
bins = [0, 5000, 10000, 15000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, np.inf]
labels = ['0-5k', '5k-10k', '10k-15k', '15k-20k', '20k-30k', '30k-40k', '40k-50k', '50k-60k', '60k-70k', '70k-80k', '80k-90k', '90k-100k', '100k+']

# Create a new column 'price_range'
vehicles_df['price_range'] = pd.cut(vehicles_df['price'], bins=bins, labels=labels, right=False)
vehicles_df.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer,price_range
33078,22995,2009.0,ford f-150,good,8.0,gas,101433.000000,automatic,truck,grey,1.0,2019-04-06,48,ford,20k-30k
279,5999,2012.0,nissan altima,excellent,4.0,gas,99000.000000,automatic,sedan,black,0.0,2018-12-27,28,nissan,5k-10k
44292,12900,1990.0,chevrolet silverado,excellent,8.0,gas,115553.461738,automatic,truck,red,1.0,2018-08-14,51,chevrolet,10k-15k
44573,3900,2005.0,ford escape,excellent,6.0,gas,153000.000000,automatic,SUV,white,1.0,2018-09-08,50,ford,0-5k
25739,7200,2013.0,toyota corolla,excellent,4.0,gas,109868.000000,automatic,sedan,unknown,0.0,2018-06-03,73,toyota,5k-10k
26114,18900,2011.0,cadillac escalade,excellent,8.0,gas,100540.000000,automatic,SUV,unknown,1.0,2019-03-12,24,cadillac,15k-20k
12772,12995,2010.0,ram 1500,good,8.0,gas,131000.000000,automatic,truck,silver,1.0,2018-06-18,9,ram,10k-15k
8924,2000,2003.0,honda accord,like new,4.0,gas,176000.000000,automatic,sedan,unknown,0.0,2018-12-18,3,honda,0-5k
10502,10995,2015.0,honda civic,excellent,4.0,gas,103000.000000,automatic,sedan,silver,0.0,2018-05-02,20,honda,10k-15k
36254,29995,2017.0,ford f250 super duty,like new,8.0,gas,42631.000000,automatic,truck,white,1.0,2019-01-17,24,ford,20k-30k


In [23]:
color_map_price = {
    '5k-10k': 'blue',
    '20k-30k': 'green',
    '0-5k': 'red',
    '10k-15k': 'gray',
    '15k-20k': 'tan',
    '30k-40k': 'black',
    '50k-60k': 'orange',
    '40k-50k': 'purple',
    '100k+': 'pink',
    '70k-80k': 'yellow',
    '60k-70k': 'brown',
    '90k-100k': 'cyan',
    '80k-90k': 'magenta'
}

fig = px.histogram(vehicles_df, x='type', color='price_range', title='Number of vehicles by price and type', color_discrete_map=color_map_price)
fig.show()

In [27]:
fig = px.scatter(vehicles_df, x='model_year', y='price', color='condition', title='Number of vehicles by price and type', color_discrete_map=color_map_price)
fig.show()

In [20]:
#example from blog post

st.header('Compare price distribution between manufacturers')
manufac_list = sorted(vehicles_df['manufacturer'].unique())
manufacturer_1 = st.selectbox('Select manufacturer 1',
                              manufac_list, index=manufac_list.index('chevrolet'))

manufacturer_2 = st.selectbox('Select manufacturer 2',
                              manufac_list, index=manufac_list.index('hyundai'))
mask_filter = (vehicles_df['manufacturer'] == manufacturer_1) | (vehicles_df['manufacturer'] == manufacturer_2)
df_filtered = vehicles_df[mask_filter]
normalize = st.checkbox('Normalize histogram', value=True)
if normalize:
    histnorm = 'percent'
else:
    histnorm = None
st.write(px.histogram(df_filtered,
                      x='price',
                      nbins=30,
                      color='manufacturer',
                      histnorm=histnorm,
                      barmode='overlay'))

## Compare price distribution between manufacturers